In [1]:
from random import sample
from scipy import stats
from scipy.stats import chisquare
from collections import defaultdict
from collections import Counter
import numpy as np
import math
import time
import sys

# Pval simulation

## Generation of random counts

The following function generates a list of `n` random numbers selected from the range `[0, 1, 2, 3]` and converts them to a list of four counts (number of occurences of `0`, `1`, `2`, `3`).

In [2]:
def generate_random_counts(n:int):
    sampled_list = []
    for _ in range(n):
        rval = sample([0, 1, 2, 3],1)
        sampled_list.append(rval[0])
    d = defaultdict(int)
    d[0] = 0; d[1] = 0; d[2] = 0; d[3] = 0;
    for x in sampled_list:
        d[x] += 1
    random_counts = []
    for k,v in d.items():
        random_counts.append(v)
    return random_counts

In [3]:
random_counts = generate_random_counts(1000)
random_counts

[259, 236, 236, 269]

This version of the function is simpler and more efficient, but still not efficient enough.

In [4]:
def generate_random_counts_2(n:int):
    sampled_list = list(np.random.choice([0, 1, 2, 3], size=n, p=[0.25,0.25,0.25,0.25]))
    x = Counter(sampled_list)
    return [x[0], x[1], x[2], x[3]]

In [5]:
random_counts_2 = generate_random_counts_2(1000)
random_counts_2

[241, 250, 247, 262]

### Generation of multiple random counts

The two functions above generate random counts for just one interactions, calling a numpy sampling method each time. Calling such methods in Python is expensive. Therefore it is better to make only a few method calls and draw as many many random numbers as possible. 

In [6]:
# Number of simulated interactions
i_num = 100000

# Number of reads of each interaction
rp_num = 15

Time consumption of the first method:

In [7]:
start = time.time()

random_counts_list = []
for i in range(0, i_num):
    random_counts_list.append(generate_random_counts(rp_num))
    
end = time.time()
print('Time: ' + str(end - start))

Time: 3.7967689037323


Time consumption of the second improved method:

In [8]:
start = time.time()

random_counts_list = []
for i in range(0, i_num):
    random_counts_list.append(generate_random_counts_2(rp_num))
    
end = time.time()
print('Time: ' + str(end - start))

Time: 4.16831111907959


Method with few calls of numpy random sampling methodes:

In [9]:
start = time.time()

# Get population of read pair types (0, 1, 2, 3)
population_size = 1000000
population = np.random.choice([0, 1, 2, 3], size=population_size, p=[0.25,0.25,0.25,0.25])

end = time.time()
print('Time: ' + str(end - start))

Time: 0.04401516914367676


In [10]:
def count_four_types(random_sample):
    indices, counts = np.unique(random_sample, return_counts=True)
    random_counts = np.array([0, 0, 0, 0])
    random_counts[indices] = counts
    return random_counts    

start = time.time()

# Draw index samples for all interactions at once
multi_sample_idx = np.random.randint(population_size - 1, size=(i_num, rp_num))
#print(population[multi_sample_idx])

end = time.time()
print('Time: ' + str(end - start))
start = time.time()

# Determine read type counts for each simulated interaction
random_counts_list = []
for random_sample in population[multi_sample_idx]: # Runtime could be further improved with 'apply'
    indices, counts = np.unique(random_sample, return_counts=True)
    random_counts = np.array([0, 0, 0, 0])
    random_counts[indices] = counts
    random_counts_list.append(random_counts)

# Alternative solutions with list comprehension and map (does not save any time)
#random_counts_list = [count_four_types(random_sample) for random_sample in population[multi_sample_idx]]
#random_counts_list = list(map(count_four_types, population[multi_sample_idx]))
    
end = time.time()
print('Time: ' + str(end - start))
#print(random_counts_list[:10])

Time: 0.014905691146850586
Time: 1.9816792011260986


## Simple/Twisted binomial P-value

The following function takes a list of four random counts, sums the counts for `0` and `1` and calculates a binomial P-value.

In [11]:
def binom12(random_counts, verbose=False):
    N = sum(random_counts)
    K = random_counts[0] + random_counts[1]
    pval = stats.binom_test(K, n=N, p=0.5, alternative='two-sided')
    if verbose:
        print('Randomly generated list of four counts: ' + str(random_counts))
        print('Number of successes (K): ' + str(K))
        print('Number of trials (N): ' + str(N))
    return pval

In [12]:
pval = binom12(generate_random_counts_2(20), verbose=True)
print('Binomial-01 P-value: ' + str(pval))

Randomly generated list of four counts: [5, 5, 4, 6]
Number of successes (K): 10
Number of trials (N): 20
Binomial-01 P-value: 1.0


## Highest two counts binomial P-value

The function above takes a list of four random counts, sums the highest two counts, and calculates a binomial P-value.

In [149]:
def binomHT(random_counts, verbose=False):
    N = sum(random_counts)
    sorted_counts = sorted(random_counts)
    K = sum(sorted_counts[2:])
    pval = stats.binom_test(K, n=N, p=0.5, alternative='greater')
    if verbose:
        print('Randomly generated list of four counts: ' + str(random_counts)) 
        print('Sorted four counts: ' + str(sorted_counts))
        print('Number of successes (K): ' + str(K))
        print('Number of trials (N): ' + str(N))   
    return pval

In [150]:
pval = binomHT(generate_random_counts_2(20), verbose=True)
print('Binomial-HT P-value: ' + str(pval))

Randomly generated list of four counts: [3, 8, 5, 4]
Sorted four counts: [3, 4, 5, 8]
Number of successes (K): 13
Number of trials (N): 20
Binomial-HT P-value: 0.13158798217773438


## Chi-squared P-value

The following function takes a list of four random counts and calculates a chi-squared P-value.

In [15]:
def chisq_pval(random_counts, verbose=False):
    pval = stats.chisquare(f_obs=random_counts)[1]
    if verbose:
        print('Randomly generated list of four counts: ' + str(random_counts))     
    return pval

In [16]:
pval = chisq_pval(generate_random_counts_2(20), verbose=True)
print('Chi-square P-value: ' + str(pval))

Randomly generated list of four counts: [3, 5, 7, 5]
Chi-square P-value: 0.6593898197119847


## Problem with chi-square test and small counts

The [documementation for the chi-square](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chisquare.html) test states that the test is not valid if the counts are too small:

*This test is invalid when the observed or expected frequencies in each category are too small. A typical rule is that all of the observed and expected frequencies should be at least 5. According to [3], the total number of samples is recommended to be greater than 13, otherwise exact tests (such as Barnard’s Exact test) should be used because they do not overreject.*

This text was taken from the documentation of the chi-square goodness of fit test. However, Barnard's exact test is a test of independence that makes use of a contingency table to determine the independence of two factors. Therfore, Barnard's exact test is not adequate for our problem of defining unbalanced interactions. The text above should be placed in the documentation for the [chi-square test with contingency table](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2_contingency.html).

For the data we are analyzing, there are many interactions with counts smaller than 5 and there are also interactions with total read counts that are smaller than 13, especially for Hi-C data. We therefore use the chi-square goodness of fit test only here for comparison.

## Comparison of different P-values

We have a function that randomly generates a list of four random counts and four different ways of computing a P-value. The following code generates a certain number of random count lists and, for each list, all four P-values are calculated and added to lists. At the end, the number of P-values are determined that are below a specified P-value threshold. Because the lists were randomly generated, a certain number of tests can be expected to be signifcant. This number depends on the number of iterations and the P-value threshold.

In [196]:
rp_total = 20        # Total number of reads per interaction
iter_num = 2000     # Number of simulated counts quadruples
pval_thresh = 0.05   # P-value threshold

print("Total number of reads per interaction: " + "{:,}".format(rp_total))
print("Number of simulations: " + "{:,}".format(iter_num))
print("P-value threshold: " + "{:.5f}".format(pval_thresh))

# Lists of P-values for all simulated quadruples
simplePvals = []
htPvals = []
chi2_pvals = []

# Expected number of significant tests given the P-value threshold and the number of simuulations
exp_nsig = int(iter_num*pval_thresh)
print("Expected number of times 'significant': " + "{:,}".format(exp_nsig))
print()

# Perform simualtions and calculate P-values of the different tests
for _ in range(iter_num):
    
    # Get random quadruple
    random_counts = generate_random_counts_2(rp_total)
    
    # Simple/Twisted    
    pval = binom12(random_counts)
    simplePvals.append(pval)
    
    # Heaviest two
    pvalHT = binomHT(random_counts)
    htPvals.append(pvalHT)
    
    # Chi-square godness of fit    
    pvalChi2 = chisq_pval(random_counts)
    chi2_pvals.append(pvalChi2)

# Determine obeserved numbers of significant tests and compare them with the expected value
st_nsig = sum([x<pval_thresh for x in simplePvals])
fold_change = -exp_nsig/st_nsig if st_nsig < exp_nsig else st_nsig/exp_nsig
print("ST pval score times 'significant': " + str(st_nsig) + "\n\tActual threshold: " + "{:.5f}".format(st_nsig/iter_num) + "\n\tFold change: " + "{:.2f}".format(fold_change))
print()
ht_nsig = sum([x<pval_thresh for x in htPvals])
fold_change = -exp_nsig/ht_nsig if ht_nsig < exp_nsig else ht_nsig/exp_nsig
print("HT pval score times 'significant': " + str(ht_nsig) + "\n\tActual threshold: " + "{:.5f}".format(ht_nsig/iter_num) + "\n\tFold change: " + "{:.2f}".format(fold_change))
print()
cq_nsig = sum([x<pval_thresh for x in chi2_pvals])
fold_change = -exp_nsig/cq_nsig if cq_nsig < exp_nsig else cq_nsig/exp_nsig
print("Chi-square pval times 'significant' : " + str(cq_nsig) + "\n\tActual threshold: " + "{:.5f}".format(cq_nsig/iter_num) + "\n\tFold change: " + "{:.2f}".format(fold_change))
print()

Total number of reads per interaction: 20
Number of simulations: 2,000
P-value threshold: 0.05000
Expected number of times 'significant': 100

ST pval score times 'significant': 83
	Actual threshold: 0.04150
	Fold change: -1.20

HT pval score times 'significant': 257
	Actual threshold: 0.12850
	Fold change: 2.57

Chi-square pval times 'significant' : 88
	Actual threshold: 0.04400
	Fold change: -1.14



**P-value threshold: `0.05` and `20,000` simulations**
```
Total number of reads per interaction: 11
Number of simulations: 20,000
P-value threshold: 0.05000
Expected number of times 'significant': 1,000

ST pval score times 'significant': 221
	Actual threshold: 0.01105
	Fold change: -4.52

HT pval score times 'significant': 3697
	Actual threshold: 0.18485
	Fold change: 3.70

Chi-square pval times 'significant' : 845
	Actual threshold: 0.04225
	Fold change: -1.18
```

With the ST binomial model, the null hypothesis is rejected too often. The observed number of significant tests is about **five times smaller** than expected. With the HT binomial model, the null hypothesis is clearly under-rejected. The observed number of significant tests is **more than three times larger** than expected. With the chi-square godness of fit test, the observed number of significant tests is close to the expected number. The test is slightly over-rejecting the null hypothesis.

**P-value threshold: `0.01` and `20,000` simulations**
```
Total number of reads per interaction: 11
Number of simulations: 20,000
P-value threshold: 0.01000
Expected number of times 'significant': 200

ST pval score times 'significant': 23
	Actual threshold: 0.00115
	Fold change: -8.70

HT pval score times 'significant': 652
	Actual threshold: 0.03260
	Fold change: 3.26

Chi-square pval times 'significant' : 109
	Actual threshold: 0.00545
	Fold change: -1.83
```

With the ST binomial model and a more stringent P-value threhold, the null hypothesis is rejected even more often. The observed number of significant tests is about **nine times smaller** than expected. With the HT binomial model, the null hypothesis is again clearly under-rejected. The observed number of significant tests is still **more than three times larger** than expected. With this more stringent P-value threshold, the chi-square godness of fit test is over-rejecting the null hypothesis. The observed number of significant tests is only half as large than expected.

**P-value threshold: `0.01` and `100,000` simualtions**
```
Total number of reads per interaction: 11
Number of simulations: 100,000
P-value threshold: 0.01000
Expected number of times 'significant': 1,000

ST pval score times 'significant': 92
	Actual threshold: 0.00092
	Fold change: -10.87

HT pval score times 'significant': 3560
	Actual threshold: 0.03560
	Fold change: 3.56

Chi-square pval times 'significant' : 537
	Actual threshold: 0.00537
	Fold change: -1.86
```

Increasing the number of simualtions has little effect on the results.


**P-value threshold: `0.005` and `200,000` simulations**

```
Total number of reads per interaction: 11
Number of simulations: 200,000
P-value threshold: 0.00500
Expected number of times 'significant': 1,000

ST pval score times 'significant': 195
	Actual threshold: 0.00097
	Fold change: -5.13

HT pval score times 'significant': 584
	Actual threshold: 0.00292
	Fold change: -1.71

Chi-square pval times 'significant' : 967
	Actual threshold: 0.00483
	Fold change: -1.03
```

We set the P-value thresholds for a given FDR threshold by randomizing interactions. That means we do not control the type I error for each individual test, but the rate of type I errors for multiple tests via the FDR. In our current analyzes, no P-value thresholds that were determined in this way were greater than `0.005`. For such stringent P-value thresholds, we do not observe that the HT model over-rejects the null hypothesis. In fact, the opposite is true, the observed number of significant interactions is even lower than the expected number.

### Conclusion

The chi-square goodness-of-fit would be the most adequate test for our problem if we could assume that the counts for each category are greater than 5 and the sum of all counts greater than 13. According to the documentation, the test is considered invalid of this is not the case.

For the data we analyze, we cannot make these assumptions.

For the stringent P-value thresholds resulting from our FDR procedure, the number of significant simulated counts for the HT P-value score is below the expexted number.

Interestingly, although the sum of our simulated random counts was only 11, so that the conditions for the tests can never be satisfied, the number of significant simulations was always the closest to the expected number.

## Adequacy of chi-square goodness-of-fit test

The chi-square **goodness-of-fit** could be adequate for our problem. Here are the results for the same read counts we used for the Barnard's exact tests above.

In [53]:
print(stats.chisquare([10, 0, 0, 0])) # configuration 0X
print(stats.chisquare([0, 10, 0, 0])) # configuration 1X
print(stats.chisquare([0, 0, 10, 0])) # configuration 2X
print(stats.chisquare([0, 0, 0, 10])) # configuration 3X
print()
print(stats.chisquare([10, 0, 10, 0])) # configuration 02
print(stats.chisquare([10, 0, 0, 10])) # configuration 03
print()
print(stats.chisquare([0, 10, 10, 0])) # configuration 12
print(stats.chisquare([0, 10, 0, 10])) # configuration 13
print()
print(stats.chisquare([10, 10, 0, 0])) # configuration 01
print(stats.chisquare([0, 0, 10, 10])) # configuration 23

Power_divergenceResult(statistic=30.0, pvalue=1.3800570312932553e-06)
Power_divergenceResult(statistic=30.0, pvalue=1.3800570312932553e-06)
Power_divergenceResult(statistic=30.0, pvalue=1.3800570312932553e-06)
Power_divergenceResult(statistic=30.0, pvalue=1.3800570312932553e-06)

Power_divergenceResult(statistic=20.0, pvalue=0.00016974243555282632)
Power_divergenceResult(statistic=20.0, pvalue=0.00016974243555282632)

Power_divergenceResult(statistic=20.0, pvalue=0.00016974243555282632)
Power_divergenceResult(statistic=20.0, pvalue=0.00016974243555282632)

Power_divergenceResult(statistic=20.0, pvalue=0.00016974243555282632)
Power_divergenceResult(statistic=20.0, pvalue=0.00016974243555282632)


Everything is absolutely symmetrical, which is good! There is one problem though. Interactions where the majority of read counts are of one particular type, get much smaller P-values than interactions where the majority of read counts are of only two types. This also applies if the total number of reads is the same and even if the interaction with the less extreme configuration has twice as many reads. 

In [139]:
print(stats.chisquare([10, 0, 0, 0])) # configuration 0X
print(stats.chisquare([5, 0, 5, 0]))  # configuration 02
print(stats.chisquare([10, 0, 10, 0]))  # configuration 02

Power_divergenceResult(statistic=30.0, pvalue=1.3800570312932553e-06)
Power_divergenceResult(statistic=10.0, pvalue=0.01856613546304325)
Power_divergenceResult(statistic=20.0, pvalue=0.00016974243555282632)


So far had only two counts. There were exactly two most extreme cases, in each of which one or the other count is zero. These two cases are also symmetric, meaning they yield the same P-value. For a given P-value threshold, we defined interactions as powered that have enough reads to be significant in the most extreme case. We determined the minimum number of reads required for significance by setting one count to zero and incrementing the other count. At each step, we calculated the P-value. If the P-value exeeded the threshold, we used the number of reads from the previous step to define powered interactions.

With the new model using four counts, the most extreme case is where the majority of reads are of the same type. We just saw that the P-values for such configurations are much smaller than for configurations where the majority of reads are of two different types, even if the total number of reads is the same or twice as many. This results in a much larger number of powered interactions, which on average have fewer reads per interaction.

Another even bigger problem is that the chi-square test requires that the counts in each category should be at least 5. For the analyzed data, this is often not the case. Therefore, we tried Barnard's exact test, but it turned out that this test is not adequate to define unbalanced interactions.

## Minimum number of reads required for powered interactions when using chi-square

In [24]:
for i in range(1,10,1):
    increasing_counts = [i, 0, 0, 0]
    print(increasing_counts)
    pval = chisq_pval(increasing_counts)
    print(pval)
    if sys.float_info.min * sys.float_info.epsilon < pval:
        print(-math.log10(pval))
    else:
        print('INF')

[1, 0, 0, 0]
0.3916251762710877
0.4071293966335873
[2, 0, 0, 0]
0.11161022509471268
0.9522960159827726
[3, 0, 0, 0]
0.02929088653488826
1.5332674835118683
[4, 0, 0, 0]
0.007383160505359769
2.1317576901804918
[5, 0, 0, 0]
0.0018166489665723214
2.740728983869122
[6, 0, 0, 0]
0.0004398496528388289
3.356695746469692
[7, 0, 0, 0]
0.00010527618177149762
3.977669874733538
[8, 0, 0, 0]
2.4979977724652003e-05
4.6024079532343025
[9, 0, 0, 0]
5.887355583577668e-06
5.230079732945775


## Precision of P-values

There is a limit to the precision that we must must consider in our code.

In [274]:
-math.log10(sys.float_info.min * sys.float_info.epsilon)

323.3062153431158